In [1]:
# === ZERO-INSTALL, SINGLE-CELL RUN ===
# Paths
CSV_PATH = "/kaggle/input/heilmittel-bvb/diagnoseliste_extracted.csv"
SRC_ENGINE = "/kaggle/input/heilmittel-bvb/rule_engine.py"
DST_ENGINE = "/kaggle/working/rule_engine.py"

# 1) Ensure engine is importable
import shutil, os, pandas as pd
from datetime import date
shutil.copy(SRC_ENGINE, DST_ENGINE)

from rule_engine import RuleRow, PatientContext, evaluate_patient  # now import works

# 2) Load + clean CSV, build rules
def load_rules(csv_path: str, version_hint: str = "2025-01-01"):
    df = pd.read_csv(csv_path)

    # Ensure expected columns exist with safe defaults
    cols = ["icd","title","group","eligibility","requires_second_icd",
            "second_icd_hint","acute_window_months","notes","source_url","source_version"]
    for c in cols:
        if c not in df.columns:
            if c == "requires_second_icd":
                df[c] = False
            elif c == "acute_window_months":
                df[c] = pd.NA
            else:
                df[c] = ""

    # Normalize & de-NaN strings so UI text doesn't show "nan"
    df["icd"] = df["icd"].astype(str).str.upper().str.strip()
    for c in ["title","group","second_icd_hint","notes","source_url","source_version","eligibility"]:
        df[c] = df[c].astype(str).replace({"nan":"", "NaN":""}).fillna("")

    # Eligibility normalization
    df["eligibility"] = df["eligibility"].str.upper().str.strip()
    ok = {"BVB","LHB","NONE",""}
    df.loc[~df["eligibility"].isin(ok), "eligibility"] = "NONE"
    df.loc[df["eligibility"].eq(""), "eligibility"] = "NONE"

    # Booleans & ints
    df["requires_second_icd"] = df["requires_second_icd"].map(
        lambda v: str(v).strip().lower() in {"1","true","t","yes","y"}, na_action="ignore"
    )
    df["acute_window_months"] = pd.to_numeric(df["acute_window_months"], errors="coerce").astype("Int64")
    df["source_version"] = df["source_version"].replace("", version_hint)

    # Build rules dict
    rules = {r["icd"]: RuleRow(**r) for r in df.to_dict(orient="records")}
    # Quick QA
    print(
        "Rows:", len(df),
        "| BVB:", (df["eligibility"]=="BVB").sum(),
        "| LHB:", (df["eligibility"]=="LHB").sum(),
        "| NONE:", (df["eligibility"]=="NONE").sum()
    )
    return df, rules

df, rules = load_rules(CSV_PATH)

# 3) Helpers (inline)
def normalize_icds(s: str):
    import re
    toks = re.split(r"[,\s;]+", (s or "").strip())
    return [t.upper() for t in toks if t]

def icd_family(icd: str, all_codes, k: int = 20):
    stem = icd[:4] if (len(icd) >= 4 and icd[3]==".") else icd[:3] + "."
    fam = [x for x in sorted(all_codes) if x.startswith(stem)]
    return fam[:k]

# 4) --- EDIT inputs here and re-run the whole cell ---
icd_input = "R26.2 G35 I63.9"
acute_event_date = None  # e.g., date(2025, 5, 10)
# ----------------------------------------------------

patient_icds = normalize_icds(icd_input)
ctx = PatientContext(icds=patient_icds, acute_event_date=acute_event_date)

results = evaluate_patient(ctx, rules, today=date.today())

print("ICDs in:", ", ".join(patient_icds) or "—")
bvb = [r.icd for r in results if r.eligible and r.kind == "BVB"]
lhb = [r.icd for r in results if r.eligible and r.kind == "LHB"]
print("BVB:", ", ".join(bvb) or "—")
print("LHB:", ", ".join(lhb) or "—")
print("-"*60)

for r in results:
    # robust text (avoid 'nan' in output)
    title = (getattr(r, "explain", "") or "")
    badge = "🟢" if r.eligible else "⚪️"
    print(f"{badge} {r.icd} — {r.explain}")
    print("  Bedingungen:", r.conditions_met)
    print("  Fehlend   :", ", ".join(r.missing) if r.missing else "—")
    print("  Version   :", r.source_version)
    fam = icd_family(r.icd, df["icd"].tolist(), k=12)
    if fam:
        display(df[df["icd"].isin(fam)][["icd","title","eligibility","requires_second_icd","acute_window_months"]])
    print("-"*60)
df.describe()
df.head()

Rows: 420 | BVB: 0 | LHB: 0 | NONE: 420
ICDs in: R26.2, G35, I63.9
BVB: —
LHB: —
------------------------------------------------------------
⚪️ R26.2 — R26.2 – Diagnose: qualifiziert nicht
  Bedingungen: {'is_listed': False}
  Fehlend   : —
  Version   : 2024-01-01


,icd,title,eligibility,requires_second_icd,acute_window_months
351,R26.0,,NONE,False,<NA>
352,R26.1,,NONE,False,<NA>
353,R26.2,,NONE,False,<NA>


------------------------------------------------------------
⚪️ I63.9 — I63.9 – Diagnose: qualifiziert nicht
  Bedingungen: {'is_listed': False}
  Fehlend   : —
  Version   : 2024-01-01


,icd,title,eligibility,requires_second_icd,acute_window_months
155,I63.0,,NONE,False,<NA>
156,I63.1,,NONE,False,<NA>
157,I63.2,,NONE,False,<NA>
158,I63.3,,NONE,False,<NA>
159,I63.4,,NONE,False,<NA>
160,I63.5,,NONE,False,<NA>
161,I63.6,,NONE,False,<NA>
162,I63.8,,NONE,False,<NA>
163,I63.9,,NONE,False,<NA>


------------------------------------------------------------


,icd,title,group,eligibility,requires_second_icd,second_icd_hint,acute_window_months,notes,source_url,source_version
0,B94.1,,,NONE,False,,<NA>,,local-file:Heilmittel_Diagnoseliste_Stand_01_2...,2024-01-01
1,C00,-,,NONE,False,,<NA>,,local-file:Heilmittel_Diagnoseliste_Stand_01_2...,2024-01-01
2,C70.0,,,NONE,False,,<NA>,,local-file:Heilmittel_Diagnoseliste_Stand_01_2...,2024-01-01
3,C70.1,,,NONE,False,,<NA>,,local-file:Heilmittel_Diagnoseliste_Stand_01_2...,2024-01-01
4,C70.9,,,NONE,False,,<NA>,,local-file:Heilmittel_Diagnoseliste_Stand_01_2...,2024-01-01
